# 8.4.6 SGD precondicionado

**SGD precondicionado:** $\;$ introduce una **matriz de precondicionado o precondicionador** $\mathbf{M}_t$, típicamente definida positiva
$$\boldsymbol{\theta}_{t+1}=\boldsymbol{\theta}_t-\eta_t\mathbf{M}_t^{-1}\boldsymbol{g}_t$$

**Precondicionares diagonales:** $\;$ se suelen escoger por simplicidad computacional, posiblemente sin información de segundo orden

## 8.4.6.1 AdaGrad

**AdaGrad (adaptive gradient):** $\;$ propuesto para objetivos convexos y gradientes con muchos elementos nulos
$$\theta_{t+1,d}=\theta_{t,d}-\eta_t\frac{1}{\sqrt{s_{t,d}+\epsilon}}\,g_{t,d}
\qquad\text{con}\quad%
\epsilon>0\quad\text{y}\quad%
s_{t,d}=\sum_{\tau=1}^tg_{\tau,d}^2$$

**AdaGrad en notación vectorial:** $\quad\displaystyle\Delta\boldsymbol{\theta}_t=-\eta_t\frac{1}{\sqrt{\boldsymbol{s}_t+\epsilon}}\boldsymbol{g}_t$

**AdaGrad como SGD precondicionado:** $\quad\mathbf{M}_t=\operatorname{diag}(\boldsymbol{s}_t+\epsilon)\;$ y $\;\eta_t=\eta_0$

**Inconveniente:** $\;$ reduce el factor de aprendizaje (efectivo) muy rápidamente a causa del $\boldsymbol{s}_t$ creciente

## 8.4.6.2 RMSProp y AdaDelta

**RMSProp:** $\;$ en lugar de sumar gradientes cuadráticos pasados, emplea una media móvil ponderada exponencialmente (EWMA)
$$s_{t+1,d}=\beta s_{t,d}+(1-\beta)g_{t,d}^2
\qquad\text{con $\beta\approx 0.9$ para dar más peso a los gradientes recientes}$$

**RMS viene de "root mean square":** $\quad\displaystyle\sqrt{s_{t,d}}\approx\operatorname{RMS}(\boldsymbol{g}_{1:t,d})=\sqrt{\frac{1}{t}\sum_{\tau=1}^tg_{\tau,d}^2}$

**RMSProp en notación vectorial (igual que AdaGrad):** $\quad\displaystyle\Delta\boldsymbol{\theta}_t=-\eta_t\frac{1}{\sqrt{\boldsymbol{s}_t+\epsilon}}\boldsymbol{g}_t$

**AdaDelta:** $\;$ variante de RMSProp que también incluye una EWMA de las actualizaciones $\boldsymbol{\delta}_t$
$$\Delta\boldsymbol{\theta}_t%
=-\eta_t\frac{\sqrt{\boldsymbol{\delta}_{t-1}+\epsilon}}{\sqrt{\boldsymbol{s}_t+\epsilon}}\boldsymbol{g}_t
\qquad\text{con}\quad%
\boldsymbol{\delta}_t=\beta\boldsymbol{\delta}_{t-1}+(1-\beta)(\Delta\boldsymbol{\theta}_t)^2$$

**Factor de aprendizaje en AdaDelta:** $\;\eta_t=1\,$ usualmente ya que no tiene mucho efecto

## 8.4.6.3 Adam

**Adam (adaptative moment estimation):** $\;$ RMSProp (EWMA de gradientes cuadráticos) combinado con momentum (EWMA de gradientes)
$$\begin{align*}
\mathbf{M}_t&=\beta_1\mathbf{M}_{t-1}+(1-\beta_1)\boldsymbol{g}_t%
&&\text{con $\,\beta_1=0.9\,$ usualmente}\\%
\boldsymbol{s}_t&=\beta_2\boldsymbol{s}_{t-1}+(1-\beta_2)\boldsymbol{g}_t^2%
&&\text{con $\,\beta_2=0.999\,$ y $\,\epsilon=10^{-6}\,$ usualmente}
\end{align*}$$

**Actualización Adam:** $\quad\displaystyle\Delta\boldsymbol{\theta}_t=-\eta_t\frac{1}{\sqrt{\boldsymbol{s}_t}+\epsilon}\mathbf{M}_t\quad$ (con $\,\eta_t=0.001\,$ usualmente)

**Corrección de sesgo:** $\;$ con $\,\mathbf{M}_0=\boldsymbol{s}_0=\boldsymbol{0},\,$ los estimadores iniciales son pequeños, por lo que se recomienda corregir sesgos con
$$\hat{\mathbf{M}}_t=\frac{\mathbf{M}_t}{1-\beta_1^t}%
\qquad\text{y}\qquad%
\hat{\boldsymbol{s}}_t=\frac{\boldsymbol{s}_t}{1-\beta_2^t}$$

**RMSProp como caso particular:** $\;$ con $\,\beta_1=0\,$ y sin corrección de sesgo

## 8.4.6.4 Problemas con los factores adaptativos

**Ajuste de $\eta_0$:** $\;$ con métodos de escalado diagonal, del tipo $\,\eta_0\mathbf{M}_t^{-1}$

**No convergencia:** $\;$ incluso en problemas convexos a causa de los gradientes ruidosos

**Posibles soluciones:** $\;$ AMSGrad, Padam, Yogi, etc.

## 8.4.6.5 Matrices de precondicionado no diagonales

**Full-matrix Adagrad:** $\;$ usa la matriz de precondicionado
$$\mathbf{M}_t=\Bigl[(\mathbf{G}_t\mathbf{G}_t^t)^{\tfrac{1}{2}}+\epsilon\mathbf{I}_D\Bigr]^{-1}%
\quad\text{con}\quad\mathbf{G}_t=[\boldsymbol{g}_t,\dotsc,\boldsymbol{g}_1]%
\quad\text{y $\;\boldsymbol{g}_i\,$ el gradiente hallado en el paso $i$}$$

**Algoritmo shampoo:** $\;$ aproxima $\mathbf{M}$ con una matriz diagonal por bloques y explota su estructura para invertirla eficientemente